###  power  plant dataset

#####  Read file / data ingestion

In [1]:
import pandas   as  pd
import numpy as  np
import  seaborn as sns
import matplotlib.pyplot as  plt

In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
path  =  r"D:\datasets\PowerPlant (1).csv"

In [4]:
df  =  pd.read_csv(path)

####  basic data quality  checks

In [5]:
df.head()

,AT,V,AP,RH,PE
0,8.34,40.77,1010.84,90.01,480.48
1,23.64,58.49,1011.40,74.20,445.75
2,29.74,56.90,1007.15,41.91,438.76
3,19.07,49.69,1007.22,76.79,453.09
4,11.80,40.66,1017.13,97.20,464.43


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      9568 non-null   float64
 1   V       9568 non-null   float64
 2   AP      9568 non-null   float64
 3   RH      9568 non-null   float64
 4   PE      9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB


In [7]:
df.isnull().sum()

AT    0
V     0
AP    0
RH    0
PE    0
dtype: int64

In [8]:
df.duplicated().sum()

np.int64(41)

In [9]:
df.drop_duplicates(keep  =  'first' ,  
ignore_index  =  True  ,  inplace  =  True)

In [10]:
df.shape

(9527, 5)

In [11]:
df.columns

Index(['AT', 'V', 'AP', 'RH', 'PE'], dtype='object')

####  seperate  x  and  y 

In [12]:
x  =  df.drop('PE' , axis =  1)
y =  df['PE']